In [1]:
#libraries
import re
import random
import copy
from matplotlib.image import imread
import matplotlib.pyplot as plt
import numpy

#Parameters de la propuesta
B = 500 #B: Number of evaluations
alpha = 0.25 #alpha: weight of FS
beta = 0.25 #beta: weight of FM
gamma = 0.25 #gamma: weight of FC
dta = 0.25 #delta: weight of ColorDistance

pSwapInt = 0.5
pSwapOut = 0.5

#Clase Clip: Contiene toda la información de un clip. 
class clip:
    ix: int #unique index to access in all the data management
    name: str='default'
    V_dX: float #Normalized Average Velocity in X dimension
    V_dY: float #Normalized Average Velocity in Y dimension
    NFrames: float #Normalized number of frames in clip
    CF: float #Normalized Colorfulness 
    Sat: float #Normalized saturation of the frame with the highest saturation value.
    def __init__(self, ix, name, V_dX, V_dY, NFrames, CF, Sat):
        self.ix=ix
        self.name = name
        self.V_dX = V_dX
        self.V_dY = V_dY
        self.NFrames = NFrames
        self.CF = CF
        self.Sat = Sat

def generateSummaryClip(ACl,cs,pathSS,pathout):
    v_slice = [] 
    for i in range(0,Bars,1):
        image = imread(pathSS + ACl[cs[i]].name + ".png") 
        v_slice.append(image)
    
    plt.figure()
    f, axarr = plt.subplots(1,Bars,figsize=(Bars*3,3))
    for i in range(0,Bars,1):
        axarr[i].axis('off')
        axarr[i].imshow(v_slice[i])
    plt.tight_layout()
    plt.savefig(pathout)
    plt.close()

#Readfile: función para leer archivo de datos sobre Clips.
def readfile(path):
    N = 0
    K = 0
    Cont = 0
    Name = ""
    Vnorm_dX = 0
    Vnorm_dY = 0
    Vnorm_n = 0
    Vnorm_CF = 0
    Vnorm_maxSat = 0
    Clips = []
    ix = 0
    MatDist = []
    KColsDist = []
    Fin = []
    with open(path, 'r') as f:
        # Read each line in the file
        mylist = f.read().splitlines() 

        CCol = 0
        COL = 0
        ixCol = 1
        for line in mylist:
            if Cont <= 1:
                columns = re.split('=', line)
                if Cont == 0:
                    N = int(columns[1])
                else:
                    K = int(columns[1])
            else:
                UB = N + 1
                if Cont <= UB:
                    Name, Vnorm_dX, Vnorm_dY,Vnorm_n, Vnorm_CF ,Vnorm_maxSat= line.split(' ')
                    #print(Name, Vnorm_dX, Vnorm_dY,Vnorm_n, Vnorm_CF ,Vnorm_maxSat)
                    cc = clip(ix, Name, float(Vnorm_dX), float(Vnorm_dY), float(Vnorm_n), float(Vnorm_CF), float(Vnorm_maxSat))
                    Clips.append(cc)
                    ix = ix + 1 #indice unico del video
                else:
                    LB = ixCol*(N + 1) + 2
                    UB = LB + N
                    #print(Cont,LB,UB)
                    if Cont >= LB and Cont < UB:
                        #print("leo fila")
                        MatDist.append(re.split(' ', line))
                        CCol = CCol + 1
                    if CCol == N:
                        #print("Reinicio")
                        COL = COL + 1 # iterador de 3 colores
                        KColsDist.append(MatDist)
                        ixCol = ixCol + 1 #indice de color en el archivo
                        CCol = 0
                        MatDist = []
            if ((COL)%3) == 0 and COL > 0:
                #print("guardo 3 colores")
                Fin.append(KColsDist)
                MatDist = []
                KColsDist = []
                COL = 0
                
            Cont = Cont + 1
    return N,K,Clips,Fin

# Lectura archivo de música
def readMusicClipsFile(path):
    Bars = 0
    Cont = 0
    Times = []
    Tensile = []
    Diameter = []
    CentroidDiff = []
    with open(path, 'r') as f:
        mylist = f.read().splitlines() 
        #for line in mylist:
        columns = re.split('=', mylist[0])
        Bars = int(columns[1])
        for i in range(0,4,1):
            LB = Bars*i
            for j in range(0,Bars,1):
                ix = LB + j + 1
                if i == 0:
                    Times.append(float(mylist[ix]))
                if i == 1:
                    Tensile.append(float(mylist[ix]))
                if i == 2:
                    Diameter.append(float(mylist[ix]))
                if i == 3:
                    CentroidDiff.append(float(mylist[ix]))
        Cont = Cont + 1
    return Bars,Times,Tensile,Diameter,CentroidDiff

def initialSol():
    Allw = list(range(0, N, 1))
    VecFS = []
    VecFM = []
    VecFC = []
    CSol = []
    FSaux = 0
    FMaux = 0
    FCaux = 0
    for i in range(0,Bars,1):
        UB = len(Allw)-1
        iA = random.randint(0,UB)
        ix = Allw[iA]
        del(Allw[iA])
        CSol.append(ix)
        fs_bar = Diameter[i] - Clips[ix].Sat
        fm_bar = Diameter[i] - (Clips[ix].V_dX + Clips[ix].V_dY)/2
        fc_bar = Diameter[i] - Clips[ix].CF
        FSaux = FSaux + fs_bar
        FMaux = FMaux + fm_bar
        FCaux = FCaux + fc_bar
        VecFS.append(fs_bar)
        VecFM.append(fm_bar)
        VecFC.append(fc_bar)
    return Allw, CSol, VecFS, VecFM, VecFC, FSaux, FMaux,FCaux

#Actualización atributos mus-vs-vid para swap
def DeltaDifMus(solcand,ix_old,ix_new,Clips,Allw):
#Modificar el valor de FS, FM, FC - el ix_old es el indice a trabajar en la solucion
    vold = solcand[ix_old]
    vnew = Allw[ix_new]
    #nuevas distancias entre el diameter y propiedades del clip
    dSat = Diameter[ix_old] - Clips[vnew].Sat
    dMov = Diameter[ix_old] - (Clips[vnew].V_dX + Clips[vnew].V_dY)/2
    dCol = Diameter[ix_old] - Clips[vnew].CF

    return abs(dSat),abs(dMov),abs(dCol) #Algunas de estas podrían ser negativas
    

def DeltaDifMusSwapInside(solcand,ix_a,ix_b,Clips,VecFSa,VecFMa,VecCFa):
    v_a = solcand[ix_a]
    v_b = solcand[ix_b]

    restS = 0
    restM = 0
    restC = 0
    
    #nuevas distancias entre el diameter y propiedades del clip
    dSat_ab = Diameter[ix_a] - Clips[v_b].Sat
    #print("dSat_ab",dSat_ab)
    #print("VecFSa[ix_a]",VecFSa[ix_a])
    dSat_ba = Diameter[ix_b] - Clips[v_a].Sat
    #print("dSat_ba",dSat_ba)
    #print("VecFSa[ix_b]",VecFSa[ix_b])
    
    dMov_ab = Diameter[ix_a] - (Clips[v_b].V_dX + Clips[v_b].V_dY)/2
    #print("dMov_ab",dMov_ab)
    #print("dMov_ab",VecFMa[ix_a])
    dMov_ba = Diameter[ix_b] - (Clips[v_a].V_dX + Clips[v_a].V_dY)/2
    #print("dMov_ba",dMov_ba)
    #print("VecFMa[ix_b]",VecFMa[ix_b])
    
    dCol_ab = Diameter[ix_a] - Clips[v_b].CF
    dCol_ba = Diameter[ix_b] - Clips[v_a].CF

    restS = abs(VecFSa[ix_a]) + abs(VecFSa[ix_b])
    restM = abs(VecFMa[ix_a]) + abs(VecFMa[ix_b])
    restC = abs(VecCFa[ix_a]) + abs(VecCFa[ix_b])

    
    dSat = abs(dSat_ab) + abs(dSat_ba)
    #print("dSat",dSat)
    dMov = abs(dMov_ab) + abs(dMov_ba)
    #print("dMov",dMov)
    dFC = abs(dCol_ab) + abs(dCol_ba)
    return restS,restM,restC,dSat,dMov,dFC,dSat_ab,dSat_ba,dMov_ab,dMov_ba,dCol_ab,dCol_ba #Evaluate the amount of distance difference, independently of the sign

#Evaluación completa por color
def EvalALLPredColor(solcand,K,KColsDist,Bars): #indices KColsDist [i:k][l:RGB][col k][col j]
    Dist = 0
    #print(solcand)
    #print(Bars)
    for i in range(0,K,1): 
        for j in range (1,Bars,1): #recorriendo la solución, vamos de i a i-1
            k = j - 1 #indice previo, evalúo la distancia de k a j
            for l in range(0,3,1): #Uno por Channel
                #print(KColsDist[i][l][k][j])
                val_k = solcand[k]
                val_j = solcand[j]
                #print("dist de k->" + str(val_k) + " a j->" + str(val_j) + " es " + KColsDist[i][l][val_k][val_j])
                Dist = Dist + float(KColsDist[i][l][val_k][val_j])
                #print(Dist)

    return Dist


    
def SwapInside(solcand,ix_a, ix_b, QDist, Bars,KColsDist,K):
    sc = copy.copy(solcand)
    UB = Bars - 1
    Caso = 0
    flag = 0
    
    v_a = sc[ix_a]
    v_b = sc[ix_b]
    sc[ix_a] = v_b
    sc[ix_b] = v_a
    
    QDist = EvalALLPredColor(sc,K,KColsDist,Bars)

    return sc,QDist

#Methods for operators related to SwapOut

def RestaDistanciaColor(KColsDist, Q, K, caso, ix_old, ix_new, sc):
    nxt = ix_old + 1
    prv = ix_old - 1
    delt = 0
    for i in range(0,K,1): 
        for l in range(0,3,1):
            if caso == 0: 
                v_old = sc[ix_old]
                v_nxt = sc[nxt]
                delt = delt + float(KColsDist[i][l][v_old][v_nxt]) #necesito el valor en la posición, no el indice
            elif caso == 1:
                v_old = sc[ix_old]
                v_prv = sc[prv]
                delt = delt + float(KColsDist[i][l][v_prv][v_old])
            else:
                v_old = sc[ix_old]
                v_prv = sc[prv]
                v_nxt = sc[nxt]
                #print("dist de prv->" + str(v_prv) + " a v_old->" + str(v_old) + " es " + KColsDist[i][l][v_prv][v_old])
                #print("dist de old->" + str(v_old) + " a v_nxt->" + str(v_nxt) + " es " + KColsDist[i][l][v_old][v_nxt])
                delt = delt + float(KColsDist[i][l][v_prv][v_old])
                delt = delt + float(KColsDist[i][l][v_old][v_nxt])      
    return delt

def SumaDistanciaColor(KColsDist, Q, K, caso, ix_new, sc):
    nxt = ix_new + 1
    prv = ix_new - 1
    delt = 0
    for i in range(0,K,1): 
        for l in range(0,3,1):
            if caso == 0: 
                v_new = sc[ix_new]
                v_nxt = sc[nxt]
                delt = delt + float(KColsDist[i][l][v_new][v_nxt]) #necesito el valor en la posición, no el indice
            else:
                if caso == 1:
                    v_new = sc[ix_new]
                    v_prv = sc[prv]
                    delt = delt + float(KColsDist[i][l][v_prv][v_new])
                else:
                    v_new = sc[ix_new]
                    v_prv = sc[prv]
                    v_nxt = sc[nxt]
                    delt = delt + float(KColsDist[i][l][v_prv][v_new])
                    delt = delt + float(KColsDist[i][l][v_new][v_nxt])     
    return delt

#Evaluación de modificar un video en la solución por uno que está en Allowed.
#ix_old: indice en solcand que contiene el clip a quitar
#ix_new: indice en Allowed que contiene el clip a incorporar 
#Casos: (0) ix_old es el primer bar, (1) ix_old es el último bar, (3) es cualquier otro

def DeltaPredColor(solcand,K,KColsDist,Bars,ix_old,ix_new,Allowed,QDist): #suponemos que el indice nuevo ix_new está en Allowed
    sc = copy.copy(solcand) #haremos cambios sobre sc
    alw = copy.copy(Allowed) 

    #print(sc)
    #print(alw)
    nxt = ix_old + 1
    prv = ix_old - 1
    #print("next->" + str(nxt))
    #print("prv->" + str(prv))
    #valores
    v_old = sc[ix_old]
    #print("vold->" + str(v_old))
    v_new = alw[ix_new]
    #print("vold->" + str(v_new))
    if ix_old == 0: # caso = 0  - ix_old es el primer bar
        caso = 0
        #print("Caso 0")
    else: 
        if nxt == Bars: #caso = 1 - ix_old es el último bar
            caso = 1
            #print("Caso 1")
        else:
            #necesitamos quitar prev y nxt
            caso = 2
            #print("Caso 2")
        #Resta
        Dq = RestaDistanciaColor(KColsDist, QDist, K, caso, ix_old, ix_new, sc)
        #print("Antes Q-->" + str(Q))
        #print("le voy a quitar" + str(Dq))
        QDist = QDist - Dq
        #Actualizo en Allowed y en SolCand
        #del(sc[ix_old])
        del(alw[ix_new])
        alw.append(v_old)
        sc[ix_old] = v_new
        
        #Suma
        Dq = SumaDistanciaColor(KColsDist, Q, K, caso, ix_old, sc)
        QDist = QDist + Dq
        #print("Después Q-->" + str(Q))
        #print("le sumé" + str(Dq))
    return sc,alw,QDist

def ChooseObjective(rndVal,pDist,pSat,pMov):
    Opcion = 0
    UB2 = pDist + pSat
    UB3 = UB2 + pMov
    if rndVal <= pDist :
        Opcion = 0 #Minimize distance between predominant colors
    elif rndVal <= UB2 :
        Opcion = 1 #Minimize the distance between Tension and Saturation values
    elif rndVal <= UB3 :
        Opcion = 2 #Minimize the distance between Tension and AVGMovement values
    else:
        Opcion = 3 #Minimize the distance between Tension and Colorfulness values
    return Opcion

def CheckAccepted(a,b):  
    if a < b:
        return True
    else:
        return False

In [3]:
#Main
#Indices de los clips en la solucion actual
CandSol = [] #vector TW x 1 - position [i] contains the clip assigned to TW [i]
Vec_KCol_Dist = [] # Position [i] has the distance in (R,G,B) channels between clip i and i+1
PopOfSolutions = []
Vec_FC = []
Vec_FS = [] # vector TW x 1 - position [i] contains the distance between Cl-Diam_i and Sat in clip assigned to TW [i]
Vec_FM = [] #vector TW x 1 - position [i] contains the distance between Cl-Diam_i and AVG(Movement) in clip assigned to TW [i]
N = 0 #number of clips
K = 0 #number of predominant colors

path = "VideoDataFinal_K3.txt"
Clips = []
KColsDist = [] 
N,K, Clips, KColsDist = readfile(path)

# KColsDist - Vector of Distances Matrices - len(Dist) = K [i:0-K-1][R/G/B][celda]
#Music Data Initialization
pathMus = "MusicTextFiles/SoulEyes.txt"
Times = []
Tensile = []
Diameter = []
CentroidDiff = []
Bars = 0

Bars,Times, Tensile, Diameter, CentroidDiff = readMusicClipsFile(pathMus)
auxNorm = Diameter
Diameter = Diameter/numpy.linalg.norm(Diameter,ord=1) 

In [4]:
#Allowed tenemos el conjunto de los videos no asignados
#CandSol tiene el set de clips elegidos

#Randomly generate the initial solution
Allowed, CandSol, Vec_FS, Vec_FM, Vec_FC, FS, FM,FC = initialSol()
FE = EvalALLPredColor(CandSol,K,KColsDist,Bars)


In [17]:
Criteria = []
for i in range(0,Bars,1):
    Criteria.append('rnd')

In [18]:
it = 0
dSat= 0
dMov = 0
dCol = 0
Opcion = 3
sc = []
alw = []
Accepted = False
#pSwapInt = 0.5
generateSummaryClip(Clips,CandSol,"SSDos/","Clip_"+str(it)+".png")
pDist = 0.20
pSat = 0.30
pMov = 0.40
#Local Search
while it < B:
    
    decOperator = random.random()
    decObjective = random.random()
    Opcion = ChooseObjective(decObjective,pDist,pSat,pMov)

    if decOperator <= pSwapInt: 
        #Hacemos Swap Interno
        ix_a = random.randrange(0, len(CandSol),1) 
        ix_b = random.randrange(0, len(CandSol),1) 
        while ix_a == ix_b:
            ix_b = random.randrange(0, len(CandSol),1)

        restS,restM,restC,dSat,dMov,dFC,dSat_A,dSat_B,dMov_A,dMov_B,dCol_A,dCol_B  = DeltaDifMusSwapInside(CandSol,ix_a,ix_b,Clips,Vec_FS,Vec_FM,Vec_FC)
        sc,Q = SwapInside(CandSol,ix_a, ix_b, FE, Bars, KColsDist,K)
        #TEST = EvalALLPredColor(sc,K,KColsDist,Bars)
        #print("TEST - SIN",TEST," obtuve:",Q)
        
        if Opcion == 0: Accepted = CheckAccepted(Q,FE)
        elif Opcion == 1: Accepted = CheckAccepted(dSat,restS)
        elif Opcion == 2: Accepted = CheckAccepted(dMov,restM)
        else: Accepted = CheckAccepted(dFC,restC)
            
        if Accepted == True:
            CandSol = copy.copy(sc)
            Criteria[ix_a] = str(Opcion)
            Criteria[ix_b] = str(Opcion)
            FE = Q
            Vec_FS[ix_a] = dSat_A
            Vec_FS[ix_b] = dSat_B
            Vec_FM[ix_a] = dMov_A
            Vec_FM[ix_b] = dMov_B
            Vec_FC[ix_a] = dCol_A
            Vec_FC[ix_a] = dCol_B
            
            FS = FS - restS + dSat
            FM = FM - restM + dMov
            FC = FC - restC + dFC
    else:
        #Hacemos Swap Externo
        ind_old = random.randrange(0, len(CandSol),1) 
        ind_new = random.randrange(0,len(Allowed),1)
        #Update Attributes Distance Tension w. Clips att.
        #CUIDADO: Primero DeltaDifMus luego DeltaPredColor
        dSat,dMov,dCol = DeltaDifMus(CandSol,ind_old,ind_new,Clips,Allowed) ## cmp with Vec_FS,Vec_FM,Vec_FC
        sc,alw,Q = DeltaPredColor(CandSol,K,KColsDist,Bars,ind_old,ind_new,Allowed,FE)
        #TEST = EvalALLPredColor(sc,K,KColsDist,Bars)
        #print("TEST - SEXT",TEST," obtuve:",Q)
        
        if Opcion == 0: Accepted = CheckAccepted(Q,FE)
        elif Opcion == 1: Accepted = CheckAccepted(dSat,Vec_FS[ind_old])
        elif Opcion == 2: Accepted = CheckAccepted(dMov,Vec_FM[ind_old])
        else: Accepted = CheckAccepted(dCol,Vec_FC[ind_old])

        if Accepted == True:
            Criteria[ind_old] = str(Opcion)
            CandSol = copy.copy(sc)
            Allowed = copy.copy(alw)
            FE = Q
            FS = FS - Vec_FS[ind_old] + dSat
            FM = FM - Vec_FM[ind_old] + dMov
            FC = FC - Vec_FC[ind_old] + dCol
            Vec_FS[ind_old] = dSat
            Vec_FM[ind_old] = dMov
            Vec_FC[ind_old] = dCol
           
    if not(it%50):
        generateSummaryClip(Clips,CandSol,"SSDos/","Clip_"+str(it)+".png")
    
    sc.clear()
    alw.clear()
    Accepted = False
    it += 1
generateSummaryClip(Clips,CandSol,"SSDos/","Clip_"+str(it)+".png")

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

Error in callback <function flush_figures at 0x1320cc720> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

In [ ]:
FE = EvalALLPredColor(CandSol,K,KColsDist,Bars)

In [ ]:
print(FE)
print(K)
print(CandSol)

In [ ]:
a = []
b = []
for cl in Clips:
    #a.append(cl.Sat)
    #mov = (cl.V_dX + cl.V_dY)/2
    a.append(cl.CF)
    #a.append(mov)
    b.append(cl.name)
ss = sorted(a,reverse=True)
print([b[a.index(i)] for i in ss]) ## gives [4, 2, 1, 0, 3]


In [ ]:
print(KColsDist[0])

In [ ]:
#REPARAR CASO EN QUE LOS INDICES SON VECINOS
def SwapInside(solcand,ix_a, ix_b, QDist, Bars,KColsDist,K):
    sc = copy.copy(solcand)
    UB = Bars - 1
    Caso = 0
    flag = 0
    #casos ix_a 
    if ix_a == 0: #a está al inicio
        if ix_b == UB:
            
            Caso = 0 #b está al final
        else:
            
            Caso = 1 #b está al medio
    else:
        if ix_a == UB: #a está al final
            if ix_b == 0: #b al principio
                
                Caso = 2
            else:
                print("SINT- caso 3")
                Caso = 3
        else: #el a está en el medio
            if ix_b == 0: 
                print("SINT- caso 4")
                Caso = 4 #b está al inicio
            elif ix_b == UB:
                print("SINT- caso 5")
                Caso = 5 #b está al final
            else:
                print("SINT- caso 6")
                Caso = 6 # ambos in between
                flag = 1
    #Resta y Suma
    #RestQ,SumQ = RestSumaDistColSwapInside(KColsDist, QDist, K, Caso, ix_a, ix_b, sc)
    #print("QDist:",QDist,"RestQ:",RestQ,"SumQ:",SumQ)
    #print("Antes Q-->" + str(QDist))
    #print("le voy a quitar" + str(RestQ))
    #print(sc)
    #QDist = QDist - RestQ
    v_a = sc[ix_a]
    v_b = sc[ix_b]
    sc[ix_a] = v_b
    sc[ix_b] = v_a
    #print(sc)
    #print("Hago el Cambio")
    #print(sc)
    #Suma
    #QDist = QDist + SumQ
    QDist = EvalALLPredColor(sc,K,KColsDist,Bars)
    #print("Después Q-->" + str(QDist))
    #print("le sumé" + str(SumQ))

    return sc,QDist
def RestSumaDistColSwapInside(KColsDist, QDist, K, Caso, ix_a, ix_b, sc):
    nxt_a = ix_a + 1
    prv_a = ix_a - 1
    
    nxt_b = ix_b + 1
    prv_b = ix_b - 1
    #print("Caso:" + str(Caso))
    #print(ix_a, sc[ix_a], ix_b, sc[ix_b])
    delt = 0
    deltSum = 0
    for i in range(0,K,1): 
        for l in range(0,3,1):
            if Caso == 0: #[a,b] = [0,UB]   
                #print("SINT- caso 0")
                #[a] = 0
                v_a = sc[ix_a]
                v_nxt_a = sc[nxt_a]
                delt = delt + float(KColsDist[i][l][v_a][v_nxt_a]) 

                #[b] = UB
                v_b = sc[ix_b]
                v_prv_b = sc[prv_b]
                delt = delt + float(KColsDist[i][l][v_prv_b][v_b])

                deltSum = deltSum + float(KColsDist[i][l][v_b][v_nxt_a]) 
                deltSum = deltSum + float(KColsDist[i][l][v_prv_b][v_a])
                
            elif Caso == 1: # [a,b] = [0,btw] #Caso Vecinos
                #print("SINT- caso 1")
                #[a] = 0
                v_a = sc[ix_a]
                v_nxt_a = sc[nxt_a]
                delt = delt + float(KColsDist[i][l][v_a][v_nxt_a]) 

                #[b] = btw
                v_b = sc[ix_b]
                v_prv_b = sc[prv_b]
                v_nxt_b = sc[nxt_b]
                delt = delt + float(KColsDist[i][l][v_prv_b][v_b])
                delt = delt + float(KColsDist[i][l][v_b][v_nxt_b])    
                
                # [b,a] = [0,btw]
                deltSum = deltSum + float(KColsDist[i][l][v_b][v_nxt_a]) 
                
                deltSum = deltSum + float(KColsDist[i][l][v_prv_b][v_a])
                deltSum = deltSum + float(KColsDist[i][l][v_a][v_nxt_b])   
                    
            elif Caso == 2: #[a,b] = [UB,0]
                #print("SINT - caso 2")
                #[a] = UB
                v_a = sc[ix_a]
                v_prv_a = sc[prv_a]
                delt = delt + float(KColsDist[i][l][v_prv_a][v_a])

                #[b] = 0
                v_b = sc[ix_b]
                v_nxt_b = sc[nxt_b]
                delt = delt + float(KColsDist[i][l][v_b][v_nxt_b]) 
                
                #[b,a] = [UB,0]
                deltSum = deltSum + float(KColsDist[i][l][v_prv_a][v_b])
                deltSum = deltSum + float(KColsDist[i][l][v_a][v_nxt_b])
                
            elif Caso == 3: #[a,b] = [UB,btw] #Caso Vecinos
                #print("SINT- caso 3")
                #[a] = UB
                v_a = sc[ix_a]
                v_prv_a = sc[prv_a]
                delt = delt + float(KColsDist[i][l][v_prv_a][v_a])

                #[b] = btw
                v_b = sc[ix_b]
                v_prv_b = sc[prv_b]
                v_nxt_b = sc[nxt_b]
                delt = delt + float(KColsDist[i][l][v_prv_b][v_b])
                delt = delt + float(KColsDist[i][l][v_b][v_nxt_b]) 
                
                #[b,a] = [UB,btw]
                deltSum = deltSum + float(KColsDist[i][l][v_prv_a][v_b])

                deltSum = deltSum + float(KColsDist[i][l][v_prv_b][v_a])
                deltSum = deltSum + float(KColsDist[i][l][v_a][v_nxt_b]) 
                 
            elif Caso == 4: #[a,b] = [btw,0] #Caso Vecinos
                #print("SINT- caso 4")
                #[a] = btw
                v_a = sc[ix_a]
                v_prv_a = sc[prv_a]
                v_nxt_a = sc[nxt_a]
                
                delt = delt + float(KColsDist[i][l][v_prv_a][v_a])
                delt = delt + float(KColsDist[i][l][v_a][v_nxt_a])     
                
                v_b = sc[ix_b]
                v_nxt_b = sc[nxt_b]
                delt = delt + float(KColsDist[i][l][v_b][v_nxt_b])
                #[a] = 0
                deltSum = deltSum + float(KColsDist[i][l][v_a][v_nxt_b])

                #[b,a] = [btw,0]
                deltSum = deltSum + float(KColsDist[i][l][v_prv_a][v_b])
                deltSum = deltSum + float(KColsDist[i][l][v_b][v_nxt_a]) 
            elif Caso == 5: #[a,b] = [btw,UB] #Caso Vecinos
                #print("SINT- caso 5")

                #[a] = btw
                v_a = sc[ix_a]
                v_prv_a = sc[prv_a]
                v_nxt_a = sc[nxt_a]
                
                delt = delt + float(KColsDist[i][l][v_prv_a][v_a])
                delt = delt + float(KColsDist[i][l][v_a][v_nxt_a])     
        
                #[b] = UB
                v_b = sc[ix_b]
                v_prv_b = sc[prv_b]
                delt = delt + float(KColsDist[i][l][v_prv_b][v_b])
                #[b,a] = [btw,UB]
                deltSum = deltSum + float(KColsDist[i][l][v_prv_b][v_a])
                
                deltSum = deltSum + float(KColsDist[i][l][v_prv_a][v_b])
                deltSum = deltSum + float(KColsDist[i][l][v_b][v_nxt_a])
                
            else:#CASO 6: [a,b] = [btw,btw] #Caso Vecinos
                #[a] = btw

                v_a = sc[ix_a]
                v_prv_a = sc[prv_a]
                v_nxt_a = sc[nxt_a]
                
                delt = delt + float(KColsDist[i][l][v_prv_a][v_a])
                delt = delt + float(KColsDist[i][l][v_a][v_nxt_a])     
        
                #print("SINT- caso 6")
                #[b] = btw
                v_b = sc[ix_b]
                v_prv_b = sc[prv_b]
                v_nxt_b = sc[nxt_b]
                #print(v_a,v_b)
                delt = delt + float(KColsDist[i][l][v_prv_b][v_b])
                delt = delt + float(KColsDist[i][l][v_b][v_nxt_b])
                
                #[b,a] = [btw,btw]
                deltSum = deltSum + float(KColsDist[i][l][v_prv_b][v_a])
                deltSum = deltSum + float(KColsDist[i][l][v_a][v_nxt_b]) 
                deltSum = deltSum + float(KColsDist[i][l][v_prv_a][v_b])
                deltSum = deltSum + float(KColsDist[i][l][v_b][v_nxt_a]) 

    #print(delt,deltSum)
    return delt,deltSum

In [ ]:
if Opcion == 0: print(it,"Dist",Accepted,FE)
    elif Opcion == 1: print(it,"Sat",Accepted,FS)
    elif Opcion == 2: print(it,"Mov", Accepted, FM)
    else:  print(it,"Colorful",Accepted,FC)